In [1]:
pip install scikit-fuzzy

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install networkx

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Keyring is skipped due to an exception: org.freedesktop.DBus.Error.NoServer: Failed to connect to socket /tmp/dbus-6SBnyjBASS: Connection refused
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import time
import cv2
from pop import LiDAR, Pilot
from pop.Util import imshow
from pop import Util



WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [4]:
# Enable image display
Util.enable_imshow()

# Initialize LiDAR and Car
lidar = LiDAR.Rplidar()
car = Pilot.AutoCar()

# Connect LiDAR and start motor
lidar.connect()
lidar.startMotor()


In [5]:
car.setSpeed(60)
car.steering = 0


In [6]:
# Define fuzzy variables
distance = ctrl.Antecedent(np.arange(0, 2000, 1), 'distance')  # Distance in mm
degree = ctrl.Antecedent(np.arange(60, 301, 1), 'degree')  # Angle in degrees (60 to 300)
speed = ctrl.Consequent(np.arange(20, 61, 1), 'speed')  # Speed: 20 (slow), 60 (fast)
turning = ctrl.Consequent(np.arange(60, 301, 1), 'turning')  # Steering: 60 to 300 degrees

In [7]:
# Fuzzy membership functions for distance
distance['close'] = fuzz.trimf(distance.universe, [0, 0, 500])  # Close distance
distance['far'] = fuzz.trimf(distance.universe, [500, 1500, 2000])  # Far distance

# Fuzzy membership functions for degree
degree['left'] = fuzz.trimf(degree.universe, [60, 90, 120])  # Left turn
degree['straight'] = fuzz.trimf(degree.universe, [120, 180, 240])  # Straight ahead
degree['right'] = fuzz.trimf(degree.universe, [240, 270, 300])  # Right turn

# Fuzzy membership functions for speed
speed['slow'] = fuzz.trimf(speed.universe, [20, 20, 40])  # Slow speed
speed['fast'] = fuzz.trimf(speed.universe, [40, 60, 60])  # Fast speed

# Fuzzy membership functions for turning (opposite direction of obstacle)
turning['left'] = fuzz.trimf(turning.universe, [60, 90, 120])  # Left turn
turning['straight'] = fuzz.trimf(turning.universe, [120, 180, 240])  # Straight
turning['right'] = fuzz.trimf(turning.universe, [240, 270, 300])  # Right turn

In [ ]:
# Define fuzzy rules
rule1 = ctrl.Rule(distance['close'] & degree['left'], (turning['left'], speed['slow']))  
rule2 = ctrl.Rule(distance['close'] & degree['right'], (turning['right'], speed['slow']))  
rule3 = ctrl.Rule(distance['far'], (turning['straight'], speed['fast']))  
rule4 = ctrl.Rule(degree['straight'] & distance['close'], (turning['left'], speed['slow']))  
rule5 = ctrl.Rule(degree['straight'] & distance['far'], (turning['straight'], speed['fast']))  

# Create control system and simulation
turning_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
turning_sim = ctrl.ControlSystemSimulation(turning_ctrl)

In [9]:
# Camera setup
cam = Util.gstrmer(width=640, height=480, fps=30, flip=0)
camera = cv2.VideoCapture(cam, cv2.CAP_GSTREAMER)

if not camera.isOpened():
    print("Camera not found")


In [ ]:
# Main loop
try:
    while True:
        # Capture camera frame
        ret, frame = camera.read()
        if not ret:
            break

        # Fetch LiDAR vectors and map
        vectors = lidar.getVectors()
        lidar_map = lidar.getMap(limit_distance=2000, size=(50, 50))
        imshow("map", lidar_map)

        # Filter LiDAR data
        filtered_vectors = [(degree, distance) for degree, distance, *_ in vectors if distance > 200]
        left_obstacle = min((dist for deg, dist in filtered_vectors if 60 <= deg <= 120), default=2000)
        right_obstacle = min((dist for deg, dist in filtered_vectors if 240 <= deg <= 300), default=2000)

        # Use fuzzy logic for decision-making
        if left_obstacle < right_obstacle:
            degree_value = 90  # Obstacle on the left
        elif right_obstacle < left_obstacle:
            degree_value = 270  # Obstacle on the right
        else:
            degree_value = 180  # No immediate obstacle, go straight

        # Assign fuzzy values based on distance and degree
        turning_sim.input['distance'] = min(left_obstacle, right_obstacle)
        turning_sim.input['degree'] = degree_value
        turning_sim.compute()

        # Get output and print the linguistic results
        decision = turning_sim.output['turning']
        speed_value = turning_sim.output['speed']

        # Convert the degree to a linguistic output (Left, Straight, Right)
        if 60 <= decision <= 120:
            turning_result = "Left"
        elif 240 <= decision <= 300:
            turning_result = "Right"
        else:
            turning_result = "Straight"

        # Print the linguistic results for steering and speed
        print(f"Steering: {turning_result}, Speed: {speed_value} ")

        # Control car speed based on fuzzy output
        car.setSpeed(speed_value)

        # Display camera frame
        cv2.imshow("Camera Feed", frame)

        # Delay for stability
        time.sleep(0.1)

except KeyboardInterrupt:
    print("Stopping Robot...")

finally:
    car.stop()
    lidar.stopMotor()
    camera.release()
    cv2.destroyAllWindows()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Steering: Straight, Speed: 51.04098336137594 


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.045551814440714 
Steering: Straight, Speed: 51.045551814440714 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.045551814440714 
Steering: Straight, Speed: 51.045551814440714 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.03641255605379 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.04098336137594 
Steering: Straight, Speed: 51.045551814440714 
Steering: Straight, Speed: 51.045551814440714 
Steering: Straight, Speed: 5